In [ ]:
%matplotlib inline
from IPython.display import clear_output
import time
import string

Q-learning agent
---

In [ ]:
# TODO LIST
# - Implement wrapper for reward engineering

In [ ]:
%run env.py
%run rl-helpers.py

# Create drones
n_drones = 3
drones_names = list(string.ascii_uppercase[:n_drones])

# Create environment
env = DirectionQTable(DeliveryDrones(drones_names))
states = env.reset()
print('Number of states:', env.observation_space.n)

# Create drones
drones = {n: RandomAgent(env.observation_space.n, env.action_space.n) for n in drones_names}
my_drone_name = drones_names[0]
my_drone = QLearningAgent(
    state_size=env.observation_space.n,
    action_size=env.action_space.n,
    gamma=0.9, alpha=0.01,
    epsilon_start=1, epsilon_end=0.01, epsilon_decay=0.98
)
drones[my_drone_name] = my_drone

# Create trainer
trainer = MultiAgentTrainer(env, drones, seed=0)

# Train for a few steps, plot results
my_drone.greedy(False)
trainer.train(5000)
plot_rolling_rewards(trainer.rewards_log)
render_qtable(my_drone.q_table, fmt='{:.2g}')

# Test agents
my_drone.greedy(True)
rewards_log = test_agents(env, drones, n_steps=1000, seed=0)
plot_cumulated_rewards(rewards_log)

Test agents
---

In [ ]:
# Make sure our drone behaves greedily
my_drone.greedy(True)

# Simulation loop
states = env.reset()
rewards = None
while True:
    # Render
    clear_output(wait=True)
    print(env.render('ainsi'))

    # Act
    actions = {name: agent.act(states[name]) for name, agent in drones.items()}

    # Print last rewards and next actions
    print('Last rewards:', rewards)
    print('Next actions:', {n: Action(i) for n, i in actions.items()})

    # Sleep, step, learn
    time.sleep(0.5)
    states, rewards, dones, _ = env.step(actions)

Benchmarking
---

In [ ]:
%run env.py
%run rl-helpers.py

# How many participants?
n_drones = 25

# Create drones & environment
drones_names = range(n_drones)
env = DirectionQTable(DeliveryDrones(drones_names))
drones = {drone_name: RandomAgent(env.observation_space.n, env.action_space.n) for drone_name in drones_names}
states = env.reset()

# Run drones
for _ in tqdm_notebook(range(10**6)):
    actions = {name: agent.act(states[name]) for name, agent in drones.items()}
    states, rewards, dones, _ = env.step(actions)

Graphics
---

In [ ]:
# TODO

# Drone, pickup, dropoff, collision ~ simple geometric shapes
# https://image.freepik.com/free-vector/simple-geometric-shapes-background_1168-371.jpg
# Animation: slightly growing/shrinking to simulate up/down movement with shade

# Desired output
# https://img.deszone.net/2018/05/simple-geometric-shapes-free-vector-pattern4.jpg
# https://as1.ftcdn.net/jpg/01/72/82/18/500_F_172821814_Oyl3cNYBcigDpeCzehbAQghLxJILrZA5.jpg

# Other ideas
# Drones leave a fading trace